In [1]:
#Standard Lib imports
import sys, os
from pathlib import Path
import getpass
from time import sleep
from tqdm.notebook import tqdm
from fastcore.utils import parallel
from fastcore.foundation import L



# Third party imports
import pandas as pd

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))

# Local application imports
from sei import sei
from selenium.webdriver.support.ui import Select

USR = getpass.getuser()
PWD = getpass.getpass("Digite sua Senha: ")
print(f'Olá {USR}')

%load_ext autoreload
%autoreload 2

Digite sua Senha:  ··········


Olá rsilva


In [2]:
FILE = Path(r"D:\OneDrive - ANATEL\GR01FI3\Denúncias\Denúncias_PRD.xlsx")
saida = Path(r'C:\Users\rsilva\Downloads\pendentes')

In [12]:
s = sei.login_sei(USR, PWD, browser='Chrome', timeout=5, teste=False, is_headless=False)

In [17]:
# s.page.driver.get('http://sistemas.anatel.gov.br/se/public/view/b/srd.php')

# s.page.driver.execute_script("toggleFilter();")

# from selenium.webdriver.common.keys import Keys
# s.page._atualizar_elemento(('id', 'fc_0'), '50407081810' + Keys.RETURN)

# s.page._clicar(('xpath', '//*[@id="aplTbl"]/tbody/tr[2]/td[1]/button'))

In [ ]:
s.mudar_lotação("GR01OR")

In [4]:
df = pd.read_excel(FILE, dtype='str', engine='openpyxl', sheet_name='Dados')

In [5]:
df = df.iloc[:, [0, 4]]
df.columns = ['Protocolo', 'Processo']

In [6]:
df = df[df.Processo.notna()]
df = df[df.Processo != ' ']
df = df[df.Processo != '']

In [13]:
for row in tqdm(list(df.itertuples())[::-1]):
    while True:
        try:
            s.go_to_processo(row.Processo)
            p = sei.Processo(s.page, row.Processo)
            sleep(5)
            acoes = p._get_acoes(click=False)
            if acoes.get("Reabrir Processo"):
                with p._go_to_central_frame():
                    #p.page._clica_abre_nova_janela(acoes['Reabrir Processo'])
                    p.page._clicar(acoes['Reabrir Processo'])
            break
        except TimeoutError:
            pass            

  0%|          | 0/329 [00:00<?, ?it/s]

Não foi possível acessar o frame central
Não foi possível acessar o frame central


In [20]:
row.Processo

' '

In [ ]:
#df = df[(df['Unidade'].isin(['GR01OR'])) & (df['Nível de Acesso'] == 'Público')].reset_index(drop=True)

In [27]:
df.head()

,Unidade,Processo,Documento,Link SEI,Tipo de Documento,Situação do Processo
0,GR01FI3,53500.006847/2016-07,0370249,https://sei.anatel.gov.br/sei/controlador.php?...,Carta,Concluído
1,GR01FI3,53504.000294/2020-17,6386998,https://sei.anatel.gov.br/sei/controlador.php?...,Auto,Aberto
2,GR01FI3,53504.000830/2017-89,1138974,https://sei.anatel.gov.br/sei/controlador.php?...,Ofício,Concluído
3,GR01FI3,53504.001140/2017-47,1163595,https://sei.anatel.gov.br/sei/controlador.php?...,Ofício,Concluído
4,GR01FI3,53504.001796/2019-21,3872836,https://sei.anatel.gov.br/sei/controlador.php?...,Conteúdo de Mídia,Concluído


In [20]:
df.loc[:, 'Alterado'] = 'Não'

In [22]:
def alterar_acesso(row, nivel='Restrito', hip='Informação Pessoal (Art. 31 da Lei nº 12.527/2011)'):
    s.go(row['Link SEI'])
    sleep(5)            
    p = sei.Processo(s.page, row["Processo"])
#     p.armazena_arvore()
#     keys = L(p.arvore.keys())
#     i = keys.argwhere(lambda x:row['Documento'] in x)[0]
#     key = keys[i]
#     with p._go_to_arvore():
#         if p.page.check_element_exists(('id', f'anchorNA{p.arvore[key]["id"].split("anchor")[-1]}')):
#             #p.page.driver.maximize_window()
#             p.page.driver.save_screenshot(f"{saida}/{row['Processo'].replace('/', '')}.png")
#             return True
        
    sleep(5)
            
    acoes = p._get_acoes(click=False)
    if acoes.get("Reabrir Processo"):
        with p._go_to_central_frame():
            #p.page._clica_abre_nova_janela(acoes['Reabrir Processo'])
            p.page._clicar(acoes['Reabrir Processo'])
                        
        s.go(row['SEI'])
        p = sei.Processo(s.page, row["Processo"])
        sleep(5)
    
    
    acao = acoes.get("Consultar/Alterar Documento")
    if not acao:
        acao = acoes.get("Consultar/Alterar Documento Externo")
    if acao:
        with p._go_to_central_frame():
            #with p.page._clica_abre_nova_janela(acao):
            p.page._clicar(acao)
            p.page._clicar(('xpath', f'//*[@id="opt{nivel}"]'))                
            if nivel == 'Restrito':
                p.page._selecionar_por_texto(('xpath', '//*[@id="selHipoteseLegal"]'), hip)                                  
            p.page._clicar(('xpath', '//*[@id="btnSalvar"]'))
            return True    
    return False

In [8]:
rows = list(df.sample(frac=1).iterrows())

In [ ]:
#parallel(alterar_acesso, rows, pause=1)

In [33]:
for i, row in tqdm(rows):
    if alterar_acesso(row, hip=r'Fiscalização / Investigação da Anatel (Art. 174 da Lei nº 9.472/1997)'):
        df.loc[i, 'Alterado'] = 'Sim'
    else:
        df.loc[i, 'Alterado'] = 'Erro'
        print(f"Erro: {row['Processo']}")

Erro: 53504.005695/2016-87
Erro: 53504.003582/2016-47
Erro: 53500.006847/2016-07
Erro: 53504.005298/2016-13
Erro: 53504.001140/2017-47
Erro: 53504.002270/2017-05
Erro: 53504.003684/2016-62
Erro: 53504.008890/2016-69
Erro: 53504.005256/2017-55
Erro: 53504.001796/2019-21
Erro: 53504.005013/2016-36
Erro: 53504.007241/2016-41
Erro: 53504.013360/2016-32
Erro: 53504.006695/2017-85
Erro: 53504.005245/2017-75
Erro: 53504.003968/2016-59
Erro: 53504.005221/2016-35
Erro: 53504.005626/2016-73
Erro: 53504.005470/2018-92
Erro: 53504.003550/2016-41
Erro: 53504.011168/2016-10
Erro: 53504.003273/2016-77
Erro: 53504.004244/2016-22
Erro: 53504.003112/2017-64
Erro: 53504.004511/2016-61
Erro: 53504.000830/2017-89
Erro: 53504.005877/2016-58
Erro: 53504.004758/2016-88
Erro: 53504.005060/2016-80
Erro: 53504.005256/2017-55



In [ ]:
df[df.Alterado == 'Erro'].to_excel(r'C:\Users\rsilva\Downloads\erro3.xlsx', index=False)

In [ ]:
concluir_processos = df.loc[(df.Alterado == 'Sim') & (df['Situação do Processo'] == 'Concluído')]['Processo'].unique()

In [ ]:
for proc in concluir_processos:
    p = s.go_to_processo(proc)
    sleep(5)
    p.concluir_processo()

In [29]:
for i, row in df[(df.Alterado == 'Erro') & (df['Situação do Processo'] == 'Aberto')].iterrows():
    p = s.go_to_processo(row['Processo'])
    sleep(5)
    acoes = p._get_acoes(click=False)
    if acoes.get("Remover Sobrestamento do Processo"):
        with p._go_to_central_frame():
            #p.page._clica_abre_nova_janela(acoes['Reabrir Processo'])
            p.page._clicar(acoes['Remover Sobrestamento do Processo'])
            df.loc[i, 'Situação do Processo'] = 'Sobrestado'

AttributeError: 'DataFrame' object has no attribute 'Alterado'

In [34]:
df[df.Processo.str.contains('/2020')]

,Unidade,Processo,Documento,Link SEI,Tipo de Documento,Situação do Processo,Alterado
1,GR01FI3,53504.000294/2020-17,6386998,https://sei.anatel.gov.br/sei/controlador.php?...,Auto,Aberto,Sim
6,GR01FI3,53504.003052/2020-85,5754270,https://sei.anatel.gov.br/sei/controlador.php?...,Carta,Sobrestado,Sim
16,GR01FI3,53504.004820/2020-18,6101573,https://sei.anatel.gov.br/sei/controlador.php?...,Conteúdo de Mídia,Aberto,Sim
17,GR01FI3,53504.004820/2020-18,6101567,https://sei.anatel.gov.br/sei/controlador.php?...,Conteúdo de Mídia,Aberto,Sim
18,GR01FI3,53504.004820/2020-18,6101551,https://sei.anatel.gov.br/sei/controlador.php?...,Carta,Aberto,Sim
29,GR01FI3,53504.005818/2020-66,6118948,https://sei.anatel.gov.br/sei/controlador.php?...,Carta,Aberto,Sim
31,GR01FI3,53504.006673/2020-11,6192035,https://sei.anatel.gov.br/sei/controlador.php?...,Ofício,Aberto,Sim
33,GR01FI3,53504.006739/2020-72,6359313,https://sei.anatel.gov.br/sei/controlador.php?...,Defesa,Aberto,Sim
35,GR01FI3,53504.007381/2020-03,6246225,https://sei.anatel.gov.br/sei/controlador.php?...,Ofício,Aberto,Sim
36,GR01FI3,53504.007520/2020-91,6321289,https://sei.anatel.gov.br/sei/controlador.php?...,Carta,Aberto,Sim


In [ ]:
for i, row in copy.iterrows():
    p = s.go_to_processo(row['Processo'])
    sleep(5)
    acoes = p._get_acoes(click=False)
    if acoes.get("Sobrestar Processo"):
        with p._go_to_central_frame():
            p.page._clicar(acoes['Sobrestar Processo'])
            p.page._clicar(('xpath', '//*[@id="optSomenteSobrestar"]'))
            p.page._atualizar_elemento(('xpath', '//*[@id="txaMotivo"]'), 'Será atendida em breve.')
            p.page._clicar(('xpath', '//*[@id="sbmSalvar"]'))

In [ ]:
sub = df[df['Situação do Processo'] == "Concluir"]
for i, row in tqdm_notebook(sub.iterrows(), total=sub.shape[0]):
    p = s.go(row['Link SEI'])
    p = sei.Processo(s.page, row["Processo"])
    #sleep(1)
    #p.concluir_processo()
    df.loc[i, "Situação do Processo"] = "OK"

In [ ]:
sub = df[df.Alterado == "Erro"]
for i, row in tqdm_notebook(sub.iterrows(), total=sub.shape[0]):
    if alterar_acesso(link=row['Link SEI'], nivel='Restrito', hip=r'Fiscalização / Investigação da Anatel (Art. 174 da Lei nº 9.472/1997)'):
        df.loc[i, 'Alterado'] = 'Sim'
    else:
        df.loc[i, 'Alterado'] = 'Erro' 
        print('Erro: {row}')

In [ ]:
df.to_csv(FILE.parent / 'Resumo_Alterados.csv', index=False)